In [1]:
import tensorflow as tf
import numpy as np
import gym

/Users/f.seiler/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


# Hyperparameters

In [2]:
env_name = "CartPole-v0"
num_envs = 3
n_steps = 50
n_actions = 2
state_size = 4
entropy_coef = 0
vf_coef = 0.1
learning_rate = 0.002
lam = 0.95
gamma = 0.99
episodes = 20000

# Make environments

In [3]:
from multiprocessing_env import SubprocVecEnv
def make_env():
    def _thunk():
        env = gym.make(env_name)
        return env

    return _thunk
env = gym.make(env_name)

In [4]:
envs = [make_env() for i in range(num_envs)]
envs = SubprocVecEnv(envs)

Process Process-2:
Process Process-3:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/f.seiler/anaconda/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/f.seiler/anaconda/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/f.seiler/anaconda/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/f.seiler/anaconda/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/f.seiler/anaconda/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/Users/f.seiler/Documents/Development/Notebooks/reinforcement_learning/multiprocessing_env.py", line 11, in worker
    cmd, data = remote.recv()
  File "/Users/f.seiler/Documents/Development/Notebooks/reinforcement_learning/multiproc

# A2C Network Architecture

In [5]:
# Input
X = tf.placeholder(tf.float32, shape=(None, state_size), name="X")
y = tf.placeholder(tf.int32, shape=(None, n_actions), name="y")
advantages = tf.placeholder(tf.float32, [None], name="advantages")
rewards = tf.placeholder(tf.float32, [None], name="rewards")

# Hidden
fc1 = tf.layers.dense(X, 32, activation=tf.nn.relu, kernel_initializer=
tf.contrib.layers.xavier_initializer())
fc2 = tf.layers.dense(fc1, 64, activation=tf.nn.relu, kernel_initializer=
tf.contrib.layers.xavier_initializer())
fc3 = tf.layers.dense(fc1, 10, activation=tf.nn.relu, kernel_initializer=
tf.contrib.layers.xavier_initializer())
fc4 = tf.layers.dense(fc3, n_actions, activation=None)
# Output
action_distribution = tf.nn.softmax(fc4)
value = tf.layers.dense(fc3, 1, activation=None, kernel_initializer=
tf.contrib.layers.xavier_initializer())

# Loss
neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc4, labels=y)
pg_loss = tf.reduce_mean(advantages * neg_log_prob)
vf_loss = tf.losses.mean_squared_error(tf.squeeze(value), rewards)

entropy = 0 # TODO: Implement entropy
loss = pg_loss - entropy * entropy_coef + vf_loss * vf_coef

# Gradient Clipping
params = tf.trainable_variables()

grads = tf.gradients(loss, params)
grads, grad_norm = tf.clip_by_global_norm(grads, 0.5)
grads = list(zip(grads, params))

# Training
trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = trainer.apply_gradients(grads)

Instructions for updating:
Use the retry module or similar alternatives.


# Tensorboard

In [6]:
writer = tf.summary.FileWriter("./tensorboard/5", )

In [7]:
reward_mean = tf.placeholder(tf.float32, name="reward_mean")
policy_loss = tf.placeholder(tf.float32, name="policy_loss")
value_loss = tf.placeholder(tf.float32, name="value_loss")
tf.summary.scalar("Reward_mean", reward_mean)
tf.summary.scalar("Policy_loss", policy_loss)
tf.summary.scalar("Value_loss", value_loss)
write_op = tf.summary.merge_all()

# Discounted sum of rewards 

In [8]:
def discount_with_dones(next_value, rewards, masks, gamma):
    r = next_value.reshape(next_value.shape[0])
    returns = []
    for step in reversed(range(len(rewards))):
        r = rewards[step] + gamma * r * masks[step]
        returns.insert(0, r)
    return returns

# Main

In [9]:
states = envs.reset()

In [10]:
print(states.shape)
with tf.Session() as sess:
    reward_mean_ = 0
    sess.run(tf.global_variables_initializer())
    for ep in range(episodes):
        mb_states, mb_actions, mb_rewards, mb_values, mb_dones, mb_masks = [],[],[],[],[], []
        for _ in range(n_steps):
            # Feed forward current states 
            action_dist, values = sess.run([action_distribution, value], {X:states})
            # Sample actions from action_dist
            action = list(map(lambda p: np.random.choice(range(2), p=p), action_dist))
            mb_states.append(states)
            
            states, reward, done, _ = envs.step(action)

            n_values = np.max(n_actions)
            mb_actions.append(np.eye(n_values)[action])

            mb_values.append(values.flatten())
            mb_dones.append(done)
            mb_rewards.append(reward)
            mb_masks.append(1-done)
        next_values = sess.run(value, {X: states})
        mb_rewards = np.asarray(mb_rewards)
        mb_returns = discount_with_dones(next_values, mb_rewards, mb_masks, gamma)
        mb_returns = np.asarray(mb_returns)

        mb_actions = np.asarray(mb_actions)
        mb_states = np.asarray(mb_states)
        mb_values = np.asarray(mb_values)

        

        mb_advs = np.asarray(mb_returns) - np.asarray(mb_values)
        mb_states = mb_states.reshape(mb_states.shape[0]*mb_states.shape[1],mb_states.shape[2])
        mb_advs = mb_advs.reshape(mb_advs.shape[0]*mb_advs.shape[1])
        mb_returns = mb_returns.reshape(mb_returns.shape[0]*mb_returns.shape[1])
        mb_actions = mb_actions.reshape(mb_actions.shape[0]*mb_actions.shape[1],mb_actions.shape[2])
        
        feed_dict = {X: mb_states,
            y: mb_actions,
            advantages: mb_advs, # Use to calculate our policy loss
            rewards: mb_returns}

        policy_loss_, value_loss_, loss_, _ = sess.run([pg_loss, vf_loss, loss, train], feed_dict)
        print("Value:",value_loss_)
        print("Policy:",policy_loss_)
        print("Loss:",loss_)
        
        if(ep % 10 == 0):
            tstate = env.reset()
            done = False
            rewards_ = []
            while not done:
                #env.render()
                action_dist, values = sess.run([action_distribution, value], {X:tstate.reshape(1,tstate.shape[0])})
                action=np.random.choice(range(2), p=action_dist[0])
                tstate, reward, done, _ = env.step(action)
                rewards_.append(reward)
            reward_mean_ = np.sum(rewards_)
        summary = sess.run(write_op, feed_dict={reward_mean: reward_mean_,policy_loss: policy_loss_, value_loss: value_loss_})
        writer.add_summary(summary, ep)
        writer.flush()

(3, 4)
Value: 245.75464
Policy: 8.395199
Loss: 32.970665
Value: 150.47705
Policy: 7.1826806
Loss: 22.230387
Value: 82.86145
Policy: 5.3332496
Loss: 13.619394
Value: 154.2622
Policy: 7.061064
Loss: 22.487286
Value: 267.16608
Policy: 9.1708765
Loss: 35.887486
Value: 120.17129
Policy: 6.202107
Loss: 18.219236
Value: 90.42116
Policy: 5.479164
Loss: 14.52128
Value: 128.8208
Policy: 6.5663157
Loss: 19.448395
Value: 95.20132
Policy: 5.8165717
Loss: 15.336704
Value: 86.41106
Policy: 5.3299103
Loss: 13.971016
Value: 125.03217
Policy: 6.133245
Loss: 18.636463
Value: 108.58234
Policy: 5.9836893
Loss: 16.841923
Value: 120.774704
Policy: 6.2184176
Loss: 18.295889
Value: 175.98384
Policy: 7.8759236
Loss: 25.474308
Value: 93.27336
Policy: 5.5913877
Loss: 14.918724
Value: 103.68042
Policy: 5.712853
Loss: 16.080894
Value: 236.6212
Policy: 8.908905
Loss: 32.571026
Value: 109.305626
Policy: 6.13585
Loss: 17.066414
Value: 326.42987
Policy: 10.35706
Loss: 43.000046
Value: 118.058395
Policy: 6.2348156
Loss:

Value: 226.98695
Policy: 4.960909
Loss: 27.659605
Value: 121.002045
Policy: 2.486369
Loss: 14.586574
Value: 214.53027
Policy: 4.358486
Loss: 25.811514
Value: 100.18242
Policy: 0.73303276
Loss: 10.751275
Value: 158.63626
Policy: 2.5092723
Loss: 18.372898
Value: 162.81325
Policy: 3.0176177
Loss: 19.298943
Value: 123.438736
Policy: 2.150335
Loss: 14.494209
Value: 181.57367
Policy: 0.8229314
Loss: 18.980299
Value: 218.21542
Policy: 4.5792527
Loss: 26.400795
Value: 140.60152
Policy: 2.220143
Loss: 16.280294
Value: 86.34979
Policy: 1.0222068
Loss: 9.6571865
Value: 78.91467
Policy: -1.0760024
Loss: 6.815465
Value: 215.21292
Policy: 4.13399
Loss: 25.655281
Value: 244.38657
Policy: 5.0803847
Loss: 29.519043
Value: 138.2612
Policy: 0.96031135
Loss: 14.786431
Value: 238.04683
Policy: 4.948737
Loss: 28.753422
Value: 112.205765
Policy: -0.56434506
Loss: 10.656231
Value: 188.48146
Policy: 3.7010849
Loss: 22.54923
Value: 273.23468
Policy: 6.1875668
Loss: 33.511036
Value: 110.55819
Policy: 1.2395238
L

Value: 236.96
Policy: 7.454585
Loss: 31.150585
Value: 248.71123
Policy: 6.43261
Loss: 31.303732
Value: 301.0423
Policy: 8.7293
Loss: 38.83353
Value: 313.88995
Policy: 0.4698487
Loss: 31.858845
Value: 256.85184
Policy: 8.443022
Loss: 34.128204
Value: 458.17517
Policy: -4.463172
Loss: 41.354343
Value: 311.2125
Policy: 3.6491013
Loss: 34.77035
Value: 415.5151
Policy: -1.7649829
Loss: 39.786526
Value: 257.0981
Policy: 4.0688214
Loss: 29.778633
Value: 284.38013
Policy: 4.098471
Loss: 32.536484
Value: 511.8736
Policy: 0.7084377
Loss: 51.895798
Value: 459.2825
Policy: -0.40660295
Loss: 45.521645
Value: 497.69223
Policy: 0.1760937
Loss: 49.945316
Value: 413.4044
Policy: -0.39768416
Loss: 40.942753
Value: 346.07733
Policy: 3.6195874
Loss: 38.22732
Value: 351.5673
Policy: 1.9647369
Loss: 37.121468
Value: 696.3389
Policy: -7.2593923
Loss: 62.374504
Value: 289.54047
Policy: 4.0981684
Loss: 33.052216
Value: 215.50085
Policy: 7.307623
Loss: 28.857708
Value: 673.5669
Policy: -8.230781
Loss: 59.125908

Value: 517.88605
Policy: -7.9265456
Loss: 43.86206
Value: 131.18977
Policy: -2.4807067
Loss: 10.638271
Value: 125.66724
Policy: 5.079094
Loss: 17.645817
Value: 208.97124
Policy: 6.275335
Loss: 27.172459
Value: 688.95233
Policy: -4.345256
Loss: 64.54998
Value: 456.89783
Policy: -3.7249157
Loss: 41.96487
Value: 131.84644
Policy: 5.604879
Loss: 18.789522
Value: 89.15043
Policy: 4.0844
Loss: 12.999443
Value: 885.34644
Policy: -9.522137
Loss: 79.01251
Value: 21.322449
Policy: 1.9428235
Loss: 4.0750685
Value: 125.74403
Policy: 6.0814347
Loss: 18.655838
Value: 161.02625
Policy: 5.1970673
Loss: 21.299692
Value: 1077.9818
Policy: -6.740002
Loss: 101.058174
Value: 597.40924
Policy: -6.445463
Loss: 53.295464
Value: 42.978527
Policy: 3.6387513
Loss: 7.9366045
Value: 572.2413
Policy: -4.396121
Loss: 52.828007
Value: 52.78937
Policy: 3.66842
Loss: 8.947357
Value: 654.8659
Policy: -3.0042987
Loss: 62.482296
Value: 201.66814
Policy: 1.1935104
Loss: 21.360325
Value: 532.50336
Policy: -5.250964
Loss: 47

Value: 346.91394
Policy: 0.74151605
Loss: 35.43291
Value: 79.605644
Policy: 4.4395456
Loss: 12.40011
Value: 821.6048
Policy: -5.2960205
Loss: 76.86446
Value: 889.6294
Policy: -7.5725446
Loss: 81.390396
Value: 391.92102
Policy: -1.6806822
Loss: 37.51142
Value: 588.52313
Policy: -2.826698
Loss: 56.025616
Value: 1339.6383
Policy: -13.429109
Loss: 120.53473
Value: 540.2488
Policy: -3.8514369
Loss: 50.173443
Value: 368.786
Policy: -0.46235684
Loss: 36.416245
Value: 67.029564
Policy: 4.275715
Loss: 10.978672
Value: 934.162
Policy: -6.1394396
Loss: 87.27676
Value: 388.32303
Policy: -0.081785224
Loss: 38.750515
Value: 530.84174
Policy: -2.083689
Loss: 51.000484
Value: 971.22394
Policy: -9.598004
Loss: 87.52439
Value: 60.800007
Policy: 3.8251417
Loss: 9.905143
Value: 93.91095
Policy: 4.953894
Loss: 14.34499
Value: 318.67178
Policy: 0.6241293
Loss: 32.491306
Value: 944.58563
Policy: -8.822335
Loss: 85.63623
Value: 59.607166
Policy: 3.8999588
Loss: 9.860676
Value: 232.65372
Policy: 1.485672
Loss:

...

Value: 11.337401
Policy: 1.5660008
Loss: 2.699741
Value: 2123.8315
Policy: -18.57527
Loss: 193.80789
Value: 14.043438
Policy: 1.7629141
Loss: 3.1672578
Value: 632.02856
Policy: -3.045253
Loss: 60.157604
Value: 19.597284
Policy: 1.8769243
Loss: 3.8366528
Value: 2027.4531
Policy: -18.095535
Loss: 184.64978
Value: 17.530355
Policy: 0.3980563
Loss: 2.1510918
Value: 723.13184
Policy: -3.0415926
Loss: 69.27159
Value: 53.846405
Policy: 2.400451
Loss: 7.7850914
Value: 1978.9841
Policy: -14.979606
Loss: 182.91882
Value: 15.242381
Policy: 1.8185114
Loss: 3.3427496
Value: 727.8869
Policy: -5.090302
Loss: 67.69839
Value: 11.27697
Policy: 1.4467479
Loss: 2.5744448
Value: 2006.6481
Policy: -20.64781
Loss: 180.017
Value: 8.56516
Policy: 1.5179571
Loss: 2.374473
Value: 17.194227
Policy: 1.6808426
Loss: 3.4002652
Value: 27.48068
Policy: 2.2841604
Loss: 5.0322285
Value: 1844.1191
Policy: -15.791073
Loss: 168.62083
Value: 2.8320658
Policy: 0.67394704
Loss: 0.9571536
Value: 7.1557627
Policy: 1.1194366
Los

Value: 17.415068
Policy: 1.6364542
Loss: 3.3779612
Value: 1205.9878
Policy: -8.129674
Loss: 112.4691
Value: 752.7762
Policy: -4.913971
Loss: 70.36365
Value: 14.686832
Policy: 1.5056609
Loss: 2.9743443
Value: 13.050257
Policy: 1.7032853
Loss: 3.008311
Value: 1363.4456
Policy: -9.733878
Loss: 126.61068
Value: 814.9404
Policy: -5.3196564
Loss: 76.174385
Value: 12.12629
Policy: 1.5501208
Loss: 2.76275
Value: 5.2359037
Policy: 0.70678633
Loss: 1.2303767
Value: 1365.588
Policy: -9.129542
Loss: 127.42927
Value: 796.3055
Policy: -5.6258917
Loss: 74.004654
Value: 14.265858
Policy: 1.9074405
Loss: 3.3340263
Value: 3.3256052
Policy: 0.76504916
Loss: 1.0976096
Value: 1403.3638
Policy: -9.510357
Loss: 130.82602
Value: 822.65717
Policy: -3.798822
Loss: 78.466896
Value: 14.326241
Policy: 1.633645
Loss: 3.0662692
Value: 8.609105
Policy: 1.3139141
Loss: 2.1748247
Value: 1412.6082
Policy: -9.365282
Loss: 131.89554
Value: 905.4424
Policy: -5.175317
Loss: 85.36893
Value: 5.301748
Policy: 1.0209775
Loss: 1

Value: 97.943565
Policy: 4.726372
Loss: 14.520728
Value: 1047.1632
Policy: -9.988328
Loss: 94.728
Value: 703.1401
Policy: -5.674113
Loss: 64.6399
Value: 44.544956
Policy: 3.0463624
Loss: 7.5008583
Value: 36.389637
Policy: 2.789858
Loss: 6.4288216
Value: 1183.761
Policy: -11.744411
Loss: 106.63168
Value: 922.7077
Policy: -6.511199
Loss: 85.759575
Value: 37.19349
Policy: 2.7208757
Loss: 6.4402246
Value: 44.461594
Policy: 3.1384156
Loss: 7.5845747
Value: 888.94824
Policy: -7.346249
Loss: 81.54858
Value: 793.92834
Policy: -6.320937
Loss: 73.0719
Value: 54.096195
Policy: 2.9014332
Loss: 8.311053
Value: 56.422638
Policy: 2.6407206
Loss: 8.282985
Value: 539.2228
Policy: -6.140492
Loss: 47.781788
Value: 367.68613
Policy: -4.7338786
Loss: 32.034733
Value: 78.3123
Policy: 2.6394439
Loss: 10.4706745
Value: 138.00374
Policy: 5.0335445
Loss: 18.83392
Value: 488.3257
Policy: -5.502313
Loss: 43.33026
Value: 312.88086
Policy: -4.1522064
Loss: 27.13588
Value: 59.597713
Policy: 2.5566661
Loss: 8.516438


Value: 755.26526
Policy: -4.862897
Loss: 70.66363
Value: 1812.4366
Policy: -15.677244
Loss: 165.56642
Value: 29.113382
Policy: 2.6020343
Loss: 5.5133724
Value: 36.4626
Policy: 2.8779094
Loss: 6.5241694
Value: 679.7709
Policy: -5.379965
Loss: 62.597122
Value: 1722.7263
Policy: -20.25843
Loss: 152.01419
Value: 24.676155
Policy: 2.0986433
Loss: 4.566259
Value: 15.426007
Policy: 1.7097199
Loss: 3.2523208
Value: 789.46423
Policy: -6.283627
Loss: 72.662796
Value: 1927.9802
Policy: -15.332195
Loss: 177.46582
Value: 19.64552
Policy: 1.7955679
Loss: 3.76012
Value: 34.79091
Policy: 2.2921994
Loss: 5.7712903
Value: 771.461
Policy: -5.386713
Loss: 71.75939
Value: 1940.485
Policy: -14.964222
Loss: 179.08429
Value: 16.732767
Policy: 1.819009
Loss: 3.4922857
Value: 34.668404
Policy: 2.5095153
Loss: 5.9763556
Value: 764.3284
Policy: -4.9802012
Loss: 71.452644
Value: 1976.9738
Policy: -16.31993
Loss: 181.37744
Value: 34.37363
Policy: 2.6579223
Loss: 6.0952854
Value: 40.38372
Policy: 3.0785959
Loss: 7.1

KeyboardInterrupt: 